In [1]:
!pip install pyaurn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 363 kB 18.0 MB/s 


In [2]:
import tensorflow as tf
import pandas as pd
import requests
import pyaurn

In [15]:
metadata = pyaurn.importMeta().set_index("site_id")

In [21]:
display(metadata)
filtered_metadata = metadata.drop(["site_name", "parameter", "Parameter_name", "start_date", "end_date", "ratified_to", "zone", "agglomeration", "local_authority"], axis=1)
site = filtered_metadata.loc[["WREX"]]
display(site)

,site_name,location_type,latitude,longitude,parameter,Parameter_name,start_date,end_date,ratified_to,zone,agglomeration,local_authority
site_id,,,,,,,,,,,,
ABD,Aberdeen,Urban Background,57.157360,-2.094278,O3,Ozone,2003-08-01,2021-09-20,2021-09-20,North East Scotland,NaN,Aberdeen City
ABD9,Aberdeen Erroll Park,Urban Background,57.157400,-2.094770,O3,Ozone,2021-10-01,ongoing,2022-06-30,North East Scotland,NaN,Aberdeen City
ABD7,Aberdeen Union Street Roadside,Urban Traffic,57.144555,-2.106472,NO,Nitric oxide,2008-01-01,ongoing,2022-06-30,North East Scotland,NaN,Aberdeen City
ABD8,Aberdeen Wellington Road,Urban Traffic,57.133888,-2.094198,NO,Nitric oxide,2016-02-09,ongoing,2022-06-30,North East Scotland,NaN,Aberdeen City
ARM6,Armagh Roadside,Urban Traffic,54.353728,-6.654558,NO,Nitric oxide,2009-01-01,ongoing,2022-06-30,Northern Ireland,NaN,Armagh
...,...,...,...,...,...,...,...,...,...,...,...,...
WRAY,Wray,Rural Background,54.104666,-2.584182,O3,Ozone,1985-04-01,1988-02-29,1988-02-29,North West & Merseyside,NaN,Lancaster
WREX,Wrexham,Urban Traffic,53.042282,-3.002829,NO,Nitric oxide,2002-03-06,ongoing,2022-06-30,North Wales,NaN,Wrexham
YW,Yarner Wood,Rural Background,50.597600,-3.716510,O3,Ozone,1987-06-26,ongoing,2022-06-30,South West,NaN,Teignbridge


,location_type,latitude,longitude
site_id,,,
WREX,Urban Traffic,53.042282,-3.002829


In [5]:
data = pyaurn.importAURN("WREX", range(2010, 2022)).reset_index()
data["site_id"] = "WREX"
display(data)

,date,NO,NO2,NOXasNO2,SO2,wd,ws,temp,site,code,PM10,PM2.5,site_id
0,2010-01-01 00:00:00,3.00000,36.00000,40.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
1,2010-01-01 01:00:00,3.00000,27.00000,31.00000,5.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
2,2010-01-01 02:00:00,4.00000,25.00000,31.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
3,2010-01-01 03:00:00,3.00000,19.00000,23.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
4,2010-01-01 04:00:00,3.00000,19.00000,23.00000,3.00000,NaN,NaN,NaN,Wrexham,WREX,NaN,NaN,WREX
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105187,2021-12-31 19:00:00,2.13558,4.26269,7.53720,0.35230,229.4,7.5,9.5,Wrexham,WREX,10.575,6.698,WREX
105188,2021-12-31 20:00:00,1.91840,5.03582,7.97732,0.28184,219.7,7.6,9.7,Wrexham,WREX,10.550,6.651,WREX
105189,2021-12-31 21:00:00,2.71472,7.17080,11.33330,0.21138,218.5,8.8,10.5,Wrexham,WREX,9.200,6.226,WREX
105190,2021-12-31 22:00:00,1.44785,4.98710,7.20710,0.07046,215.3,9.8,11.0,Wrexham,WREX,10.400,7.665,WREX


In [29]:
filtered_data = data[["date", "PM2.5", "site_id"]].dropna()
display(filtered_data)

,date,PM2.5,site_id
76681,2018-10-01 01:00:00,5.354,WREX
76682,2018-10-01 02:00:00,4.175,WREX
76683,2018-10-01 03:00:00,3.373,WREX
76684,2018-10-01 04:00:00,3.160,WREX
76685,2018-10-01 05:00:00,3.962,WREX
...,...,...,...
105187,2021-12-31 19:00:00,6.698,WREX
105188,2021-12-31 20:00:00,6.651,WREX
105189,2021-12-31 21:00:00,6.226,WREX
105190,2021-12-31 22:00:00,7.665,WREX


In [35]:
merged_data = filtered_data.merge(site, on="site_id").drop("site_id", axis=1)
display(merged_data)

,date,PM2.5,location_type,latitude,longitude
0,2018-10-01 01:00:00,5.354,Urban Traffic,53.042282,-3.002829
1,2018-10-01 02:00:00,4.175,Urban Traffic,53.042282,-3.002829
2,2018-10-01 03:00:00,3.373,Urban Traffic,53.042282,-3.002829
3,2018-10-01 04:00:00,3.160,Urban Traffic,53.042282,-3.002829
4,2018-10-01 05:00:00,3.962,Urban Traffic,53.042282,-3.002829
...,...,...,...,...,...
25891,2021-12-31 19:00:00,6.698,Urban Traffic,53.042282,-3.002829
25892,2021-12-31 20:00:00,6.651,Urban Traffic,53.042282,-3.002829
25893,2021-12-31 21:00:00,6.226,Urban Traffic,53.042282,-3.002829
25894,2021-12-31 22:00:00,7.665,Urban Traffic,53.042282,-3.002829


In [40]:
train_dataset = merged_data.sample(frac=0.8)
test_dataset = merged_data.drop(train_dataset.index)
display(train_dataset)
display(test_dataset)

,date,PM2.5,location_type,latitude,longitude
4615,2019-07-26 11:00:00,4.976,Urban Traffic,53.042282,-3.002829
17066,2020-12-28 18:00:00,32.052,Urban Traffic,53.042282,-3.002829
23903,2021-10-09 22:00:00,4.104,Urban Traffic,53.042282,-3.002829
6118,2019-09-28 02:00:00,2.642,Urban Traffic,53.042282,-3.002829
2758,2019-05-10 01:00:00,9.623,Urban Traffic,53.042282,-3.002829
...,...,...,...,...,...
23133,2021-09-07 20:00:00,12.453,Urban Traffic,53.042282,-3.002829
25401,2021-12-11 09:00:00,7.406,Urban Traffic,53.042282,-3.002829
4174,2019-07-08 01:00:00,8.609,Urban Traffic,53.042282,-3.002829
8453,2020-01-03 09:00:00,5.472,Urban Traffic,53.042282,-3.002829


,date,PM2.5,location_type,latitude,longitude
1,2018-10-01 02:00:00,4.175,Urban Traffic,53.042282,-3.002829
10,2018-10-01 11:00:00,2.877,Urban Traffic,53.042282,-3.002829
14,2018-10-01 15:00:00,3.868,Urban Traffic,53.042282,-3.002829
19,2018-10-01 20:00:00,5.000,Urban Traffic,53.042282,-3.002829
24,2018-10-02 01:00:00,0.873,Urban Traffic,53.042282,-3.002829
...,...,...,...,...,...
25854,2021-12-30 06:00:00,0.377,Urban Traffic,53.042282,-3.002829
25861,2021-12-30 13:00:00,1.769,Urban Traffic,53.042282,-3.002829
25867,2021-12-30 19:00:00,2.642,Urban Traffic,53.042282,-3.002829
25875,2021-12-31 03:00:00,1.038,Urban Traffic,53.042282,-3.002829


In [47]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop("PM2.5")
test_labels = test_features.pop("PM2.5")
train_features.shape

(20717, 4)

In [45]:
categoryLayer = tf.keras.layers.StringLookup()
categoryLayer.adapt(merged_data["location_type"])
categoryLayer.get_vocabulary()

['[UNK]', 'Urban Traffic']

In [50]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    categoryLayer,
    tf.keras.layers.Dense(units=1)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 string_lookup_2 (StringLook  (None, 4)                0         
 up)                                                             
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
